In [145]:
import tensorflow as tf
import numpy as np
import random

In [785]:
# 假设老虎机标准差为 2 倍均值
# 假设所获得的奖励符合高斯分布
BANDITS = [0.10, -0.20, 0.0, -5.0, 5.0, -20.0, 10.0, 3.0, -2.0, -1.0]
def bandit_reward(bandit):
    # 所有 bandits 的期望
    mu = BANDITS[bandit]
    sigma = np.abs(mu)*2.0
    return np.random.normal(mu, sigma, 10)

In [786]:
# 每次拉老虎机，随机从奖励结果中取一个
def pull_bandit(bandit):
    banditreward = bandit_reward(bandit)
    choose = np.random.randint(0,len(BANDITS)-1)
    return banditreward[choose]

In [787]:
# 验证一下, 可以发现基本能够达到期望值
for i, rew in enumerate(BANDITS):
    result = []
    # 重复 1000 次
    for j in range(1000):
        result.append(pull_bandit(i))
    print("Actual reward: %s; Estimated reward: %s" % (rew, np.average(result)))

Actual reward: 0.1; Estimated reward: 0.10269837746044579
Actual reward: -0.2; Estimated reward: -0.1834657223039294
Actual reward: 0.0; Estimated reward: 0.0
Actual reward: -5.0; Estimated reward: -4.775941324626682
Actual reward: 5.0; Estimated reward: 5.161489818103677
Actual reward: -20.0; Estimated reward: -17.692357382026564
Actual reward: 10.0; Estimated reward: 10.102715068772545
Actual reward: 3.0; Estimated reward: 3.229079401489951
Actual reward: -2.0; Estimated reward: -2.020107165639584
Actual reward: -1.0; Estimated reward: -0.948306270504664


In [788]:
# 贪心算法

total_reward = 0
try_reward = []
trial = 3
ite = 0
total_iter = 1000
while ite < total_iter:
    # 各 三次 实验求平均，然后以贪心算法选择最大的 Q
    if ite < trial * len(BANDITS):
        for i in range(len(BANDITS)):
            ite += 1
            reward = round(pull_bandit(i),1)
            try_reward.append(reward)
            total_reward += reward
    else:
        Q_table = np.average(np.array(try_reward).reshape(trial, len(BANDITS)), axis=0)
        # 保留两位小数
        Q_table = list(np.around(Q_table, decimals=1))
        Q = max(Q_table)
        index = Q_table.index(Q)
        ite += 1
        total_reward += Q

assert sum(try_reward) + Q*(total_iter - trial * len(BANDITS)) - total_reward < 1.0
print("获得的总收益：%d，你选择的老虎机期望收益：%s...\n" % (total_reward, BANDITS[index]))
print("你估计的 Q_table: %s, 你选择额的最大 Q: %s, %s 号老虎机。" % (Q_table, Q, index+1))
print("实际中的 Q_table: %s\n" % BANDITS)
if index == bandits.index(max(BANDITS)):
    print("You have chosen the best bandit!")
else:
    print("You haven't chosen the best bandit....")

获得的总收益：14180，你选择的老虎机期望收益：10.0...

你估计的 Q_table: [0.1, -0.3, 0.0, 0.4, 0.2, -44.8, 14.7, 6.5, -1.0, -1.7], 你选择额的最大 Q: 14.7, 7 号老虎机。
实际中的 Q_table: [0.1, -0.2, 0.0, -5.0, 5.0, -20.0, 10.0, 3.0, -2.0, -1.0]

You have chosen the best bandit!


贪心算法小结：  

注意这个结果是和 bandits 的期望以及标准差相关的。  

- 期望一定时：如果标准差很小，那么选择基本会限定在最高的几个老虎机上，如果标准差非常大，则有可能选择到其他的老虎机。
- 标准差一定时：期望相差越大，越容易选到最高的几个老虎机上，期望相差越小，则可能选到其他老虎机。

In [806]:
# epson-贪心算法

def epson_greedy(epson):
    total_reward = 0
    epson_num = 0
    try_reward = []
    trial = 3
    ite = 0
    total_iter = 1000
    while ite < total_iter:
        
        # 先三次实验确定 Q_table，再探索
        if ite < trial * len(BANDITS):
            for i in range(len(BANDITS)):
                ite += 1
                reward = round(pull_bandit(i),1)
                try_reward.append(reward)
                total_reward += reward
        
        # 上来就开始探索，第一次随机选一个
#         if ite < 1:
#             ite += 1
#             choose = np.random.randint(0,len(BANDITS)-1)
#             Q_table[choose] = round(pull_bandit(choose), 1)
        
        else:
            # 选择先 trial 次实验确定 Q_table 时，需要下面两句 计算 Q_table
            # 选择上来就开始探索时，不需要
            Q_table = np.average(np.array(try_reward).reshape(trial, len(BANDITS)), axis=0)
            Q_table = list(np.around(Q_table, decimals=1))
            
            # epson 概率选择下一个要拉的老虎机，1-epson 选目前奖励最高的 
            # 如果是非静态的 bandit，可以把 Q_table 中每台 bandit 的 reward 存成 list，然后
            # 用 Q_{n+1} = \sum_{i=1}^n \alpha (1-\alpha)^{n-i} R_i + (1-\alpha)^{n} Q_1 计算即可
            # alpha 为超参数，R_i 为存储的所有 Reward，Q1 为第一次的 Reward
            if random.random() < epson:
                epson_num += 1
                Q_table_new = Q_table
                choose = np.random.randint(0,len(BANDITS)-1)
                new_rew = pull_bandit(choose)
                # 更新的值保留两位小数，方便看结果
                Q_table_new[choose] = round(new_rew, 2)
                Q = max(Q_table_new)
                index = Q_table_new.index(Q)
            # 保留两位小数
            else:
                Q = max(Q_table)
                index = Q_table.index(Q)
            ite += 1
            total_reward += Q

    print("共探索了 %s 次." % epson_num)
    print("获得的总收益：%d，你选择的老虎机期望收益：%s...\n" % (total_reward, BANDITS[index]))
    # 如果你选的 Q（两位小数）没有在 Q_table，说明最后一次的 Q_table 是 Q_table_new，最大的 Q 是探索出来的
    # 实际中不需要 新建一个 Qtable，这里为了看起来更加直观
    print("你估计的 Q_table: %s, 你选择额的最大 Q: %s, %s 号老虎机。" % (Q_table, Q, index+1))
    print("实际中的 Q_table: %s\n" % BANDITS)
    if index == BANDITS.index(max(BANDITS)):
        print("You have chosen the best bandit!")
    else:
        print("You haven't chosen the best bandit....")
        
    return total_reward

In [817]:
# epson = 0.1，10% 的概率在探索新的老虎机
epson_greedy(0.9)

共探索了 853 次.
获得的总收益：15446，你选择的老虎机期望收益：10.0...

你估计的 Q_table: [0.1, -0.4, 0.0, 1.6, 3.9, -8.8, 21.31, 7.0, 0.2, -0.8], 你选择额的最大 Q: 21.31, 7 号老虎机。
实际中的 Q_table: [0.1, -0.2, 0.0, -5.0, 5.0, -20.0, 10.0, 3.0, -2.0, -1.0]

You have chosen the best bandit!


15446.799999999781

In [818]:
# optimistic exploration

In [819]:
# upper-confidence bound

In [820]:
# gradient